# Los Angeles Police Station Neighborhoods
This project maps police station locations and the surrounding neighborhoods colored by arrest density (2010 - present. Coffee and donut shops (along with their ratings) in the vicinity of the police stations are also shown for officer convenience. Finally, the police stations are compared and grouped by similarity of neighborhoods.

Using arrest data from Los Angeles (2010 - Present) found at https://data.lacity.org/A-Safe-City/Arrest-Data-from-2010-to-Present/yru6-6re4

In [1]:
%matplotlib inline

import sys
import types
import pandas as pd
import numpy as np
from ibm_botocore.client import Config
import ibm_boto3
import matplotlib as mpl
import matplotlib.pyplot as plt

def __iter__(self): return 0

#arrest data
client_cb48d66fda3a4f02a095fe19438f0f3b = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='',
    ibm_auth_endpoint="https://iam.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_cb48d66fda3a4f02a095fe19438f0f3b.get_object(Bucket='courseracapstonenb-donotdelete-pr-aq8s9opgnoxnuq',Key='Arrest_Data_from_2010_to_Present (1).csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

#read data from CSV file and add to dataframe
raw_arrest_data = pd.read_csv(body)
#display top rows of dataframe
raw_arrest_data.head()

,Report ID,Arrest Date,Time,Area ID,Area Name,Reporting District,Age,Sex Code,Descent Code,Charge Group Code,Charge Group Description,Arrest Type Code,Charge,Charge Description,Address,Cross Street,Location
0,111421932,07/02/2016,1845.0,14,Pacific,1411,50,M,H,24.0,Miscellaneous Other Violations,M,LAMC,LOS ANGELES MUNICIPAL CODE,BROOKS,OCEAN FRONT,"(33.9918, -118.4791)"
1,121801235,12/18/2016,940.0,18,Southeast,1822,38,F,B,24.0,Miscellaneous Other Violations,M,LAMC,LOS ANGELES MUNICIPAL CODE,300 W CENTURY BL,NaN,"(33.9456, -118.2784)"
2,150604240,01/03/2016,1315.0,6,Hollywood,669,43,M,O,24.0,Miscellaneous Other Violations,M,71.02LAMC,HIRE VEH W/O LIC,HOBART,SANTA MONICA,"(34.0908, -118.3046)"
3,150704165,04/27/2016,2230.0,7,Wilshire,721,20,F,B,6.0,Larceny,M,484(A)PC,GRAND THEFT (OVER $400),8500 BEVERLY BL,NaN,"(34.0761, -118.3766)"
4,151405507,01/22/2016,1309.0,14,Pacific,1427,60,M,W,17.0,Liquor Laws,M,25620BP,OPEN ALCOHOLIC BEV IN PUBLIC PARK/PLACE,VENICE,MOTOR,"(34.0237, -118.4246)"


## Get count of arrests by police division 
This information will be used in a choropeth map to display density of arrests per division.

In [2]:
#get count of arrests by area
raw_arrest_data.drop(['Area ID','Arrest Date','Time','Reporting District','Age','Sex Code','Descent Code','Charge Group Code','Charge Group Description','Arrest Type Code','Charge','Charge Description','Address','Cross Street','Location'], axis = 1, inplace = True)
raw_arrest_data.rename(columns={'Report ID': 'Count'}, inplace=True)
arrest_grp = raw_arrest_data.groupby('Area Name').count()
arrest_grp

,Count
Area Name,
77th Street,66711
Central,120841
Devonshire,37747
Foothill,45154
Harbor,45358
Hollenbeck,39967
Hollywood,116034
Mission,62213
N Hollywood,59825


## Get geographic coordinates of Los Angeles

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

address = 'Los Angeles, California, USA'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Los Angeles are {}, {}.'.format(latitude, longitude))

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.18.1-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00  19.46 MB/s
geopy-1.18.1-p 100% |################################| Time: 0:00:00  28.63 MB/s
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  49.95 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  26.09 MB/s
vincent-0.4.4- 100% |###################

In [4]:
# define the world map
la_map = folium.Map()

# define the world map centered around LA with a low zoom level
la_map = folium.Map(location=[latitude, longitude], zoom_start=10)

!wget --quiet http://s3-us-west-2.amazonaws.com/boundaries.latimes.com/archive/1.0/boundary-set/lapd-divisions.geojson -O  la.json
la_geo = r'la.json' # geojson file

# generate choropleth map
la_map.choropleth(
    geo_data=la_geo,
    data = raw_arrest_data,
    columns=['Area Name', 'Count'],
    key_on='feature.properties.name',
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    threshold_scale=None,
    legend_name='Arrests in LA by Division'
)

## Get location data for Los Angeles police stations & add to map
Using location data from Los Angeles police stations found at http://geohub.lacity.org/datasets/1dd3271db7bd44f28285041058ac4612_0

In [5]:
# police station location data
body = client_cb48d66fda3a4f02a095fe19438f0f3b.get_object(Bucket='courseracapstonenb-donotdelete-pr-aq8s9opgnoxnuq',Key='LAPD_Police_Stations.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_out = pd.read_csv(body)

for lat, lng, division, precinct in zip(df_out['Y'], df_out['X'], df_out['DIVISION'], df_out['PREC']):
    label = '{}, {}'.format(division, 'Precinct: ' + str(precinct))
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(la_map)  

# display map
la_map

## Get location information for each police station using the Foursquare API

In [6]:
#define API credentials
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

#get location of specific neighborhood
n_lat = df_out.loc[0, 'Y'] # neighborhood latitude value
n_long = df_out.loc[0, 'X'] # neighborhood longitude value
n_name = df_out.loc[0, 'DIVISION'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(n_name, 
                                                               n_lat, 
                                                               n_long))
#set 4sq url params and return full url
radius = 500
LIMIT = 100

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    n_lat, 
    n_long, 
    radius, 
    LIMIT)
url # display URL

results = requests.get(url).json()
#print(url)

Latitude and longitude values of HARBOR are 33.75766089658123, -118.28924155296859.


## Get venues within 500 meters of each police station

In [7]:
 # function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']# make the GET request

def getNearbyVenues(names, latitudes, longitudes):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['id'],
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue ID',
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [8]:
pd_venues = getNearbyVenues(names=df_out['DIVISION'],
                                   latitudes=df_out['Y'],
                                   longitudes=df_out['X']
                                  )

# show list of nearby venues
pd_venues.head()

HARBOR
SOUTHEAST
77TH STREET
PACIFIC
SOUTHWEST
NEWTON
CENTRAL
WEST LOS ANGELES
HOLLENBECK
WILSHIRE
OLYMPIC
RAMPART
HOLLYWOOD
NORTHEAST
NORTH HOLLYWOOD
VAN NUYS
WEST VALLEY
TOPANGA
FOOTHILL
DEVONSHIRE
MISSION


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue ID,Venue,Venue Latitude,Venue Longitude,Venue Category
0,HARBOR,33.757661,-118.289242,4bae708ef964a520a1b23be3,Fantastic Cafe,33.755438,-118.292600,American Restaurant
1,HARBOR,33.757661,-118.289242,4bd88a8635aad13a682491f3,Big Nick's Pizza,33.754244,-118.291864,Italian Restaurant
2,HARBOR,33.757661,-118.289242,4b4914a6f964a5200e6526e3,Target,33.760138,-118.293621,Big Box Store
3,HARBOR,33.757661,-118.289242,4e31c4302fb6ede816e372f6,Redbox,33.758032,-118.292398,Video Store
4,HARBOR,33.757661,-118.289242,4bc9301e3740b713a66e5e65,7-Eleven,33.758819,-118.293099,Convenience Store


## Get coffee and donut shops (and their ratings) near each police station 
This information is useful for officers to know where to get good (or bad) break snacks during working hours.


In [9]:
donuts = pd_venues.loc[pd_venues['Venue Category'] == 'Donut Shop']
coffee = pd_venues.loc[pd_venues['Venue Category'] == 'Coffee Shop']

frames = [coffee, donuts]

hangouts = pd.concat(frames)
hangouts.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue ID,Venue,Venue Latitude,Venue Longitude,Venue Category
5,HARBOR,33.757661,-118.289242,4bf808e0508c0f4777313e31,Starbucks,33.760089,-118.293582,Coffee Shop
40,PACIFIC,33.991655,-118.419842,5b82de39234724002c252318,Sachi.LA,33.991579,-118.421575,Coffee Shop
75,NEWTON,34.012356,-118.256119,4e9cb664f7905e04cd9723e8,Primera Taza,34.013348,-118.256447,Coffee Shop
111,CENTRAL,34.044019,-118.247294,4ad7ae48f964a5209d0d21e3,Spring For Coffee,34.046210,-118.250748,Coffee Shop
114,CENTRAL,34.044019,-118.247294,501b1306e4b04341bebae94c,Tierra Mia,34.044781,-118.252099,Coffee Shop


In [10]:
def get_rating(venue_id):

    vid = venue_id
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(vid, CLIENT_ID, CLIENT_SECRET, VERSION)
    #return url
    result = requests.get(url).json()

    try:
        return result['response']['venue']['rating']
    except:
        return 'None'

hangouts['Rating'] = ''

for i, row in hangouts.iterrows():
      val = get_rating(hangouts['Venue ID'][i])
      hangouts.at[i,'Rating'] = val

# get rating for each nearby coffee and donut shop
for lat, lng, name, rating in zip(hangouts['Venue Latitude'], hangouts['Venue Longitude'], hangouts['Venue'], hangouts['Rating']):
    label = '{}, {}'.format(name, 'Rating: '+ str(rating))
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='pink',
        fill=True,
        fill_color='',
        fill_opacity=0.0,
        parse_html=False).add_to(la_map)  
    
la_map

The following manipulates earlier tables into neighborhood info with latitude and longitude for clustering.

In [11]:
vlatlng = pd_venues.drop(pd_venues.columns[[3,4,5,6,7]], axis=1)
v = pd_venues.groupby('Neighborhood').count()
vlatlng.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude
0,HARBOR,33.757661,-118.289242
1,HARBOR,33.757661,-118.289242
2,HARBOR,33.757661,-118.289242
3,HARBOR,33.757661,-118.289242
4,HARBOR,33.757661,-118.289242


## One hot encoding on all venues in the vicinity of police stations for analysis of neighborhood similarities
Transform categorical information in to numeric values to be used in clustering.

In [12]:
# one hot encoding
onehot = pd.get_dummies(pd_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe

onehot['Neighborhood'] = pd_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1])
onehot = onehot[fixed_columns]

onehot.head()

,Yoga Studio,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Beer Garden,Beer Store,Big Box Store,Bookstore,Breakfast Spot,Bubble Tea Shop,Burger Joint,Bus Station,Cafeteria,Café,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Currency Exchange,Dance Studio,Dessert Shop,Diner,Dive Bar,Donut Shop,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gastropub,Gay Bar,Gift Shop,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Hobby Shop,Hookah Bar,Hot Dog Joint,Hotel,Ice Cream Shop,Indie Movie Theater,Indie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundromat,Light Rail Station,Liquor Store,Lounge,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Movie Theater,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Print Shop,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Road,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Thai Restaurant,Theater,Thrift / Vintage Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,HARBOR,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,HARBOR,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,HARBOR,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,HARBOR,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,HARBOR,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [13]:
onehot.shape

(445, 142)

In [14]:
grp = onehot.groupby('Neighborhood').mean().reset_index()
grp

,Neighborhood,Yoga Studio,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Beer Garden,Beer Store,Big Box Store,Bookstore,Breakfast Spot,Bubble Tea Shop,Burger Joint,Bus Station,Cafeteria,Café,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Currency Exchange,Dance Studio,Dessert Shop,Diner,Dive Bar,Donut Shop,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gastropub,Gay Bar,Gift Shop,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Hobby Shop,Hookah Bar,Hot Dog Joint,Hotel,Ice Cream Shop,Indie Movie Theater,Indie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundromat,Light Rail Station,Liquor Store,Lounge,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Movie Theater,Multiplex,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Print Shop,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Road,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Thai Restaurant,Theater,Thrift / Vintage Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint
0,77TH STREET,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.0,0.000000,0.000000,0.000000,0.2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,CENTRAL,0.013514,0.027027,0.027027,0.027027,0.000000,0.013514,0.000000,0.000000,0.000000,0.000000,0.000000,0.094595,0.000000,0.000000,0.000000,0.027027,0.013514,0.000000,0.027027,0.000000,0.013514,0.013514,0.000000,0.013514,0.013514,0.054054,0.000000,0.000000,0.000000,0.000000,0.013514,0.000000,0.000000,0.000000,0.013514,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013514,0.013514,0.000000,0.013514,0.013514,0.000000,0.013514,0.000000,0.000000,0.000000,0.000000,0.040541,0.013514,0.000000,0.000000,0.013514,0.013514,0.000000,0.000000,0.013514,0.000000,0.013514,0.000000,0.000000,0.040541,0.000000,0.000000,0.000000,0.013514,0.013514,0

In [15]:
grp.shape

(21, 142)

## Get frequency of venue types in each neighborhood

In [16]:
num_top_venues = 10

for hood in grp['Neighborhood']:
    print("----"+hood+"----")
    temp = grp[grp['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----77TH STREET----
                           venue  freq
0             Chinese Restaurant   0.2
1                           Food   0.2
2                     Print Shop   0.2
3                          Trail   0.2
4                     Taco Place   0.2
5                    Yoga Studio   0.0
6                      Pet Store   0.0
7        North Indian Restaurant   0.0
8  Paper / Office Supplies Store   0.0
9                           Park   0.0


----CENTRAL----
                 venue  freq
0                  Bar  0.09
1          Coffee Shop  0.05
2               Lounge  0.04
3            Gastropub  0.04
4       Ice Cream Shop  0.04
5           Restaurant  0.03
6          Art Gallery  0.03
7  Arts & Crafts Store  0.03
8             Wine Bar  0.03
9  American Restaurant  0.03


----DEVONSHIRE----
                    venue  freq
0       Fish & Chips Shop  0.15
1    Fast Food Restaurant  0.08
2        Sushi Restaurant  0.08
3       Korean Restaurant  0.08
4                    Park  0.08
5

## Find the top 10 venues near each police station

In [17]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [18]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = grp['Neighborhood']

for ind in np.arange(grp.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grp.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,77TH STREET,Print Shop,Chinese Restaurant,Trail,Taco Place,Food,Wings Joint,Food & Drink Shop,Fried Chicken Joint,French Restaurant,Food Truck
1,CENTRAL,Bar,Coffee Shop,Lounge,Ice Cream Shop,Gastropub,Burger Joint,Restaurant,Music Venue,Sushi Restaurant,Bookstore
2,DEVONSHIRE,Fish & Chips Shop,Gas Station,Sushi Restaurant,Fast Food Restaurant,Pizza Place,Park,Chinese Restaurant,Frozen Yogurt Shop,Mexican Restaurant,Donut Shop
3,FOOTHILL,Burger Joint,Fried Chicken Joint,Supermarket,Convenience Store,Sandwich Place,Food & Drink Shop,Frozen Yogurt Shop,French Restaurant,Food Truck,Food Court
4,HARBOR,Mexican Restaurant,Harbor / Marina,Convenience Store,Sandwich Place,Food Truck,Coffee Shop,Skate Park,Mobile Phone Shop,Pizza Place,Big Box Store
5,HOLLENBECK,Mexican Restaurant,Convenience Store,Food Truck,Taco Place,Grocery Store,Bakery,Japanese Restaurant,Light Rail Station,Pizza Place,Donut Shop
6,HOLLYWOOD,Coffee Shop,Pizza Place,Lounge,Vegetarian / Vegan Restaurant,Salon / Barbershop,Burger Joint,Park,Bar,Mexican Restaurant,Record Shop
7,MISSION,Fast Food Restaurant,Convenience Store,Auto Dealership,Sandwich Place,Coffee Shop,Food & Drink Shop,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck
8,NEWTON,Fast Food Restaurant,Café,Coffee Shop,Sandwich Place,Mexican Restaurant,Wings Joint,Fried Chicken Joint,French Restaurant,Food Truck,Food Court
9,NORTH HOLLYWOOD,Restaurant,Video Store,Pharmacy,Convenience Store,Tailor Shop,Wings Joint,Flower Shop,French Restaurant,Food Truck,Food Court


## Set clustering variables & set labels on venues

In [19]:
# set number of clusters
kclusters = 7

clustering = grp.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(clustering)

In [20]:
merged = grp.join(v, on='Neighborhood')
df_out = df_out.drop(df_out.columns[[2,4,5]], axis=1)
df_out = df_out.rename({'DIVISION':'Neighborhood', 'X':'Longitude', 'Y':'Latitude'}, axis=1)
#df_out.head()
# add clustering labels
merged['Cluster Labels'] = kmeans.labels_

# add latitude/longitude for each neighborhood via merged dataframes
merged = merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
merged = merged.join(df_out.set_index('Neighborhood'), on='Neighborhood')

merged.shape

(21, 162)

## Create new map with arrest density and neighborhood locations and clustering indicators

In [21]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

map_clusters.choropleth(
    geo_data=la_geo,
    data = raw_arrest_data,
    columns=['Area Name', 'Count'],
    key_on='feature.properties.name',
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    threshold_scale=None,
    legend_name='Arrests in LA by Division with Neighborhoods Colored by Similarity')
    
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(merged['Latitude'], merged['Longitude'], merged['Neighborhood'], merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

map_clusters

## Display similar locations based on assigned cluster value

In [22]:
columns = ['Neighborhood','Cluster Labels','1st Most Common Venue','2nd Most Common Venue','3rd Most Common Venue','4th Most Common Venue','5th Most Common Venue','6th Most Common Venue','7th Most Common Venue','8th Most Common Venue','9th Most Common Venue','10th Most Common Venue']
merged = merged[columns]
merged.loc[merged['Cluster Labels'] == 0, merged.columns[[0] + list(range(1, merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,CENTRAL,0,Bar,Coffee Shop,Lounge,Ice Cream Shop,Gastropub,Burger Joint,Restaurant,Music Venue,Sushi Restaurant,Bookstore
2,DEVONSHIRE,0,Fish & Chips Shop,Gas Station,Sushi Restaurant,Fast Food Restaurant,Pizza Place,Park,Chinese Restaurant,Frozen Yogurt Shop,Mexican Restaurant,Donut Shop
3,FOOTHILL,0,Burger Joint,Fried Chicken Joint,Supermarket,Convenience Store,Sandwich Place,Food & Drink Shop,Frozen Yogurt Shop,French Restaurant,Food Truck,Food Court
4,HARBOR,0,Mexican Restaurant,Harbor / Marina,Convenience Store,Sandwich Place,Food Truck,Coffee Shop,Skate Park,Mobile Phone Shop,Pizza Place,Big Box Store
5,HOLLENBECK,0,Mexican Restaurant,Convenience Store,Food Truck,Taco Place,Grocery Store,Bakery,Japanese Restaurant,Light Rail Station,Pizza Place,Donut Shop
6,HOLLYWOOD,0,Coffee Shop,Pizza Place,Lounge,Vegetarian / Vegan Restaurant,Salon / Barbershop,Burger Joint,Park,Bar,Mexican Restaurant,Record Shop
10,NORTHEAST,0,Gym / Fitness Center,Gym,Dance Studio,Martial Arts Dojo,Print Shop,Gift Shop,Furniture / Home Store,Construction & Landscaping,Paper / Office Supplies Store,Theater
12,PACIFIC,0,Mexican Restaurant,Food Truck,Sandwich Place,Café,Bus Station,Massage Studio,Donut Shop,Coffee Shop,Food,Music Store
13,RAMPART,0,Fast Food Restaurant,Mexican Restaurant,Convenience Store,Sandwich Place,Coffee Shop,Liquor Store,Steakhouse,Rental Car Location,Food Truck,Chinese Restaurant
15,SOUTHWEST,0,Fast Food Restaurant,Southern / Soul Food Restaurant,Spanish Restaurant,Donut Shop,Currency Exchange,Cosmetics Shop,Road,Sandwich Place,Seafood Restaurant,Chinese Restaurant


In [23]:
merged.loc[merged['Cluster Labels'] == 1, merged.columns[[0] + list(range(1, merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,77TH STREET,1,Print Shop,Chinese Restaurant,Trail,Taco Place,Food,Wings Joint,Food & Drink Shop,Fried Chicken Joint,French Restaurant,Food Truck


In [24]:
merged.loc[merged['Cluster Labels'] == 2, merged.columns[[0] + list(range(1, merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,MISSION,2,Fast Food Restaurant,Convenience Store,Auto Dealership,Sandwich Place,Coffee Shop,Food & Drink Shop,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck
8,NEWTON,2,Fast Food Restaurant,Café,Coffee Shop,Sandwich Place,Mexican Restaurant,Wings Joint,Fried Chicken Joint,French Restaurant,Food Truck,Food Court


In [25]:
merged.loc[merged['Cluster Labels'] == 3, merged.columns[[0] + list(range(1, merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,WEST VALLEY,3,Gym,Thrift / Vintage Store,Wings Joint,Food,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop


In [26]:
merged.loc[merged['Cluster Labels'] == 4, merged.columns[[0] + list(range(1, merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,SOUTHEAST,4,Comedy Club,Food,Gym,Gym / Fitness Center,Construction & Landscaping,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop


In [27]:
merged.loc[merged['Cluster Labels'] == 6, merged.columns[[0] + list(range(1, merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,NORTH HOLLYWOOD,6,Restaurant,Video Store,Pharmacy,Convenience Store,Tailor Shop,Wings Joint,Flower Shop,French Restaurant,Food Truck,Food Court
